# Assignment - The Battle of Neighborhoods

## 1. Set Up

### Load Libraries

In [1]:
import pandas as pd
import numpy as np

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# Install and Load Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Folium installed and imported!')

Solving environment: done

# All requested packages already installed.

Folium installed and imported!


### Read Toronto Location Data

In [3]:
# Read the saved dataframe from file
pc_table = pd.read_csv("Toronto2.csv")
#pc_table.head(10)
pc_table["Borough"].unique()

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

### Select Downtown Toronto Borough Within Toronto

In [4]:
# Select to Downtown Toronto as the borough to set up the restaurant
# There are 19 postcodes within Downtown Toronto
pc_toronto = pc_table[pc_table["Borough"].str.contains("Downtown Toronto")].reset_index(drop=True).copy()
#pc_toronto.shape
pc_toronto

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


### Show Neighborhoods In Downtown Toronto Borough

In [5]:
# Downtown Toronto's latitude and longitude coordinates
# https://latitude.to/articles-by-country/ca/canada/7404/downtown-toronto
latitude = 43.6514990607
longitude = -79.3834667995

In [6]:
# Show an initial map with neighborhoods as markers

# create map of Toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(pc_toronto['Latitude'], pc_toronto['Longitude'], pc_toronto['Borough'], pc_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
# display the map of Toronto
toronto_map

## 2. Analysis of Top Venues Within Each Neighborhood in Downtown Toronto

### Set Up FourSquare API Access

In [34]:
# Access IDs deleted

CLIENT_ID = 'Your_Client_ID' # your Foursquare ID
CLIENT_SECRET = 'Your_Client_Secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: Your_Client_ID
CLIENT_SECRET:Your_Client_Secret


### Explore Top Venues Using FourSquare

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
toronto_venues = getNearbyVenues(names=pc_toronto['Neighborhood'],
                                   latitudes=pc_toronto['Latitude'],
                                   longitudes=pc_toronto['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [10]:
print(toronto_venues.shape)
toronto_venues.head()

(1580, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Harbourfront, Regent Park",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [11]:
len(toronto_venues["Venue Category"].unique())

207

In [12]:
toronto_venues["Venue Category"].unique()

array(['Bakery', 'Coffee Shop', 'Spa', 'Restaurant', 'Historic Site',
       'Breakfast Spot', 'Farmers Market', 'Gym / Fitness Center',
       'Chocolate Shop', 'Dessert Shop', 'Park',
       'Mediterranean Restaurant', 'Mexican Restaurant', 'Pub',
       'Italian Restaurant', 'Liquor Store', 'Performing Arts Venue',
       'French Restaurant', 'Café', 'Tech Startup', 'Thai Restaurant',
       'Greek Restaurant', 'Theater', 'Shoe Store', 'Yoga Studio',
       'Animal Shelter', 'Event Space', 'Ice Cream Shop', 'Bar',
       'Cosmetics Shop', 'Food Truck', 'German Restaurant', 'Pool',
       'Art Gallery', 'Diner', 'Brewery', 'Furniture / Home Store',
       'Grocery Store', 'Middle Eastern Restaurant', 'Pet Store',
       'Pizza Place', 'Karaoke Bar', 'Sushi Restaurant', 'Gastropub',
       'Dance Studio', 'Electronics Store', 'Indian Restaurant', 'Bank',
       'Beer Store', 'Skating Rink', 'Japanese Restaurant',
       'Discount Store', 'Auto Dealership', 'Fast Food Restaurant',
    

### Exclude Neighborhoods With Under 100 Top Venues

In [14]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",42,42,42,42,42,42
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [15]:
# include only neighborhoods that have at least 100 top venues within 1km radius
g = toronto_venues.groupby('Neighborhood').count()
nb_include = g[g.Venue == LIMIT].index
toronto_venues = toronto_venues[toronto_venues['Neighborhood'].isin(nb_include)]
toronto_venues.shape

(1500, 7)

In [16]:
# 3 postcodes/neighborhoods were eliminated
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100


### Exclude Neighborhoods With Chinese Restaurants

In [17]:
# neighborhoods with Chinese restaurants in top 100 venues
toronto_venues[toronto_venues["Venue Category"].str.contains('Chinese Restaurant')].groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Bay Street,2,2,2,2,2,2
"Chinatown, Grange Park, Kensington Market",3,3,3,3,3,3
Christie,1,1,1,1,1,1
Church and Wellesley,1,1,1,1,1,1
"Harbourfront East, Toronto Islands, Union Station",1,1,1,1,1,1
"Ryerson, Garden District",1,1,1,1,1,1


In [18]:
# include only neighborhoods that have no Chinese restaurants in top 100 venues within 1km radius
g = toronto_venues[toronto_venues["Venue Category"].str.contains('Chinese Restaurant')].groupby('Neighborhood').count()
nb_include = g[g.Venue >= 0].index
toronto_venues = toronto_venues[~toronto_venues['Neighborhood'].isin(nb_include)]
toronto_venues.shape

(900, 7)

In [19]:
# 6 postcodes/neighborhoods were eliminated
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Harbord, University of Toronto",100,100,100,100,100,100
"Harbourfront, Regent Park",100,100,100,100,100,100
St. James Town,100,100,100,100,100,100
Stn A PO Boxes 25 The Esplanade,100,100,100,100,100,100


### Include Neighborhoods With Concentration of Hotels

In [20]:
toronto_venues[toronto_venues["Venue Category"]=="Hotel"].groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",5,5,5,5,5,5
Berczy Park,6,6,6,6,6,6
"Commerce Court, Victoria Hotel",8,8,8,8,8,8
"Design Exchange, Toronto Dominion Centre",10,10,10,10,10,10
"First Canadian Place, Underground city",8,8,8,8,8,8
St. James Town,5,5,5,5,5,5
Stn A PO Boxes 25 The Esplanade,5,5,5,5,5,5


In [21]:
# include only neighborhoods that have at least 8 hotels in top 100 venues within 1km radius
g = toronto_venues[toronto_venues["Venue Category"].str.contains('Hotel')].groupby('Neighborhood').count()
nb_include = g[g.Venue >= 8].index
toronto_venues = toronto_venues[toronto_venues['Neighborhood'].isin(nb_include)]
toronto_venues.shape

(300, 7)

In [22]:
# 4 postcodes/neighborhoods were eliminated
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100


## 3. Explore the Venue Mix in Remaining Neighborhoods

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Train Station,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Basketball Stadium,Beer Bar,Bistro,Bookstore,...,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater
800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
801,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
803,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
804,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(300, 68)

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Train Station,American Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Basketball Stadium,Beer Bar,Bistro,...,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater
0,"Commerce Court, Victoria Hotel",0.01,0.03,0.01,0.01,0.02,0.02,0.01,0.01,0.00,...,0.01,0.01,0.00,0.03,0.00,0.00,0.01,0.01,0.02,0.02
1,"Design Exchange, Toronto Dominion Centre",0.01,0.03,0.01,0.01,0.02,0.02,0.01,0.00,0.01,...,0.01,0.01,0.02,0.04,0.01,0.01,0.00,0.01,0.02,0.02
2,"First Canadian Place, Underground city",0.01,0.04,0.01,0.01,0.02,0.02,0.01,0.00,0.00,...,0.01,0.01,0.01,0.04,0.00,0.02,0.00,0.01,0.02,0.03


In [26]:
toronto_grouped.shape

(3, 68)

In [27]:
num_top_venues = 20

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Commerce Court, Victoria Hotel----
                  venue  freq
0                 Hotel  0.08
1           Coffee Shop  0.08
2                  Café  0.06
3            Restaurant  0.05
4    Italian Restaurant  0.04
5             Gastropub  0.04
6            Steakhouse  0.03
7   American Restaurant  0.03
8         Deli / Bodega  0.03
9          Concert Hall  0.03
10         Cocktail Bar  0.02
11      Thai Restaurant  0.02
12  Japanese Restaurant  0.02
13                  Gym  0.02
14       Farmers Market  0.02
15              Theater  0.02
16               Bakery  0.02
17                  Bar  0.02
18                  Pub  0.01
19               Lounge  0.01


----Design Exchange, Toronto Dominion Centre----
                  venue  freq
0                 Hotel  0.10
1                  Café  0.06
2           Coffee Shop  0.06
3            Steakhouse  0.04
4    Italian Restaurant  0.04
5            Restaurant  0.03
6          Concert Hall  0.03
7   American Restaurant  0.03
8         

In [28]:
#pc_toronto[pc_toronto['Neighborhood']=="Design Exchange, Toronto Dominion"]
pc_toronto2 = pc_toronto[pc_toronto['Neighborhood'].isin(toronto_grouped['Neighborhood'])]
pc_toronto2

,Postcode,Borough,Neighborhood,Latitude,Longitude
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
16,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280


## 4. Conclusion

### Summary of Remaining 3 Neighborhoods Venue Mix

* Commerce Court, Victoria Hotel -- Hotel (8), Concert Hall (3), Gym(2), Farmer's Market (2), Theatre (2)
* Design Exchange, Toronto Dominion Centre -- Hotel (10), Concert Hall (3), Park (2), Gym (2), Theatre (2)
* First Canadian Place, Underground city -- Hotel (8), Theatre (3), Concert Hall (2), Gym (2)

### Recommended Neighborhood - "Design Exchange, Toronto Dominion Centre"

Since is no distinguishable difference among the 3 remaining neighborhoods in terms of venue mix, we recommend the restaurant be set up in "Design Exchange, Toronto Dominion Centre" neighborhood.
Results from clustering this neighborhood further did not reveal clear segments. Hence, we advise an on-site visit be conducted to determine suitable premises to rent for the restaurant.

## Clustering For Further Insight

In [29]:
toronto_venues2 = toronto_venues[toronto_venues['Neighborhood']=="Design Exchange, Toronto Dominion Centre"].reset_index(drop=True).copy()
toronto_venues2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Canoe,43.647452,-79.381320,Restaurant
1,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Equinox Bay Street,43.648100,-79.379989,Gym
2,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,The Fairmont Royal York,43.645449,-79.381508,Hotel
3,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Walrus Pub & Beer Hall,43.647375,-79.379515,Pub
4,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center


In [30]:
# Let's see what happens if we cluster the neighborhoods based on latitude-longitude coordinates

# import KMeans clustering
from sklearn.cluster import KMeans

# set number of clusters to 4 (somewhat trial and error)
kclusters = 4

# drop non-numerical fields for k-means clustering
venues_cluster = toronto_venues2.drop(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude', 'Venue', 'Venue Category'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

# append cluster labels to the dataframe
toronto_venues2["Cluster_Labels"] = kmeans.labels_
toronto_venues2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster_Labels
0,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Canoe,43.647452,-79.381320,Restaurant,3
1,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Equinox Bay Street,43.648100,-79.379989,Gym,3
2,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,The Fairmont Royal York,43.645449,-79.381508,Hotel,2
3,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Walrus Pub & Beer Hall,43.647375,-79.379515,Pub,0
4,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,Adelaide Club Toronto,43.649279,-79.381921,Gym / Fitness Center,3


In [31]:
# Coordinates for "Design Exchange, Toronto Dominion Centre" neighborhood
latitude = toronto_venues2['Neighborhood Latitude'].iloc[0]
longitude = toronto_venues2['Neighborhood Longitude'].iloc[0]
print(latitude, longitude)

43.6471768 -79.38157640000001


In [32]:
# Show a revised map with the 4 detected clusters

# create map

map_clusters = folium.Map(location=[latitude,longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster, v, vcat in zip(toronto_venues2['Venue Latitude'], toronto_venues2['Venue Longitude'], toronto_venues2['Cluster_Labels'], 
                             toronto_venues2['Venue'], toronto_venues2['Venue Category']):
    label = folium.Popup(str(v) + ' -- ' + str(vcat) + ' -- ' + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

Results from applying KMeans clustering did not reveal clear segments within "Design Exchange, Toronto Dominion Centre". Hence, it is recommended that on-site visit be conducted to determine suitable premises to rent for the restaurant.